In [19]:
import os
import cv2
import torch
import numpy as np
from torchvision import models, transforms
from PIL import Image
from sklearn.cluster import KMeans
import random

In [20]:
efficientnetb3 = models.efficientnet_b3(pretrained = True)
backbone = efficientnetb3.features
backbone.eval()

c:\Users\KIIT\anaconda3\envs\video\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\KIIT\anaconda3\envs\video\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B3_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Conv2dNormActivation(
    (0): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): SiLU(inplace=True)
  )
  (1): Sequential(
    (0): MBConv(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
          (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): SiLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
          (activation): SiLU(inplace=True)
          (scale_activation): Sigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(40, 24, kernel_size=(1, 1), stride=(1, 1)

In [21]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [22]:
def extract_features(video_path, sample_rate):
    cap = cv2.VideoCapture(video_path)
    features, frames = [], []
    idx = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if idx % sample_rate == 0:  
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(rgb)
            tensor = transform(img).unsqueeze(0) 

            with torch.no_grad():
                feat = backbone(tensor) 
                feat = feat.squeeze().cpu().numpy()
            features.append(feat)
            frames.append(frame)
        idx += 1
    cap.release()
    return np.array(features), frames

In [39]:
def get_keyframes(features, frames, k):
    features = features.reshape(features.shape[0], -1) 
    kmeans = KMeans(n_clusters=k, random_state=0).fit(features)
    centers = kmeans.cluster_centers_

    keyframes = []
    keyframe_indices = []
    for center in centers:
        distances = np.linalg.norm(features - center, axis=1)
        idx = np.argmin(distances)
        keyframes.append(frames[idx])
        keyframe_indices.append(idx)
    
    keyframe_indices = sorted(set(keyframe_indices))
    keyframes = [frames[i] for i in keyframe_indices]

    return keyframes,keyframe_indices

In [40]:
video_folder = "videos/"
output_root = "keyframes2/"
os.makedirs(output_root, exist_ok=True)

In [41]:
video_files = [f for f in os.listdir(video_folder) if f.endswith((".mp4", ".avi", ".mov"))]

for n,video_file in enumerate(video_files):
    video_path = os.path.join(video_folder, video_file)
    print(f"Processing {video_file}...")

    
    features, frames = extract_features(video_path, sample_rate=10)
    if len(frames) == 0:
        print(f"⚠️ Skipped {video_file} (too short)")
        continue

    K = int(random.uniform(0.08,0.15) * len(frames))
    keyframes, indices = get_keyframes(features, frames, K)

    
    base_name = str(n+1)
    out_dir = os.path.join(output_root, base_name)
    os.makedirs(out_dir, exist_ok=True)

    
    for i, (frame, idx) in enumerate(zip(keyframes, indices)):
        out_path = os.path.join(out_dir, f"keyframe_{i+1}_frame{idx}.jpg")
        cv2.imwrite(out_path, frame)
        print(f"Saved {out_path}")

   
    txt_path = os.path.join(out_dir, f"{n+1}.txt")
    with open(txt_path, "w") as f:
        for idx in indices:
            f.write(str(idx) + "\n")

    print(f"Saved indices to {txt_path}")


Processing Air_Force_One.mp4...
Saved keyframes2/1\keyframe_1_frame3.jpg
Saved keyframes2/1\keyframe_2_frame9.jpg
Saved keyframes2/1\keyframe_3_frame13.jpg
Saved keyframes2/1\keyframe_4_frame25.jpg
Saved keyframes2/1\keyframe_5_frame38.jpg
Saved keyframes2/1\keyframe_6_frame54.jpg
Saved keyframes2/1\keyframe_7_frame68.jpg
Saved keyframes2/1\keyframe_8_frame71.jpg
Saved keyframes2/1\keyframe_9_frame89.jpg
Saved keyframes2/1\keyframe_10_frame97.jpg
Saved keyframes2/1\keyframe_11_frame99.jpg
Saved keyframes2/1\keyframe_12_frame106.jpg
Saved keyframes2/1\keyframe_13_frame124.jpg
Saved keyframes2/1\keyframe_14_frame130.jpg
Saved keyframes2/1\keyframe_15_frame137.jpg
Saved keyframes2/1\keyframe_16_frame145.jpg
Saved keyframes2/1\keyframe_17_frame157.jpg
Saved keyframes2/1\keyframe_18_frame162.jpg
Saved keyframes2/1\keyframe_19_frame166.jpg
Saved keyframes2/1\keyframe_20_frame173.jpg
Saved keyframes2/1\keyframe_21_frame181.jpg
Saved keyframes2/1\keyframe_22_frame189.jpg
Saved keyframes2/1\key

KeyboardInterrupt: 